In [1]:
"""
The initial plan was to run it on Colab to use the GPU, but we encountered an error related to the dlib library version.
Therefore, I opted to proceed locally, as this allows for better control over library versions. 
While GPU acceleration is faster, it's not strictly necessary for this study, given that we won't be working with a large dataset.

To compile dlib, I had to install Microsoft Visual Studio with C++ support.

I created a virtual environment and installed the following packages:
pip install dlib
pip install --upgrade setuptools
pip install git+https://github.com/ageitgey/face_recognition_models
pip install face_recognition
pip install opencv-python

To reproduce this on another machine, simply use the requirements.txt file."
"""
import face_recognition
import cv2
import os

d:\IA\MachineLearningStudy\05_FaceRecognition\venv\Lib\site-packages\face_recognition_models\__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [2]:
# Training the face recognition model with a dataset of known faces.

photo_paula = face_recognition.load_image_file("training\\photo_paula.jpg")
photo_ricardo = face_recognition.load_image_file("training\\photo_ricardo.jpg")
photo_kate = face_recognition.load_image_file("training\\photo_kate.jpg")

# Extracts face encodings (feature vectors) from the input image.
# Each encoding represents the distinctive features of a face.
# As our training dataset has only one face per image, we access the first (and only) encoding using index [0].
encoding_paula = face_recognition.face_encodings(photo_paula)[0]
encoding_ricardo = face_recognition.face_encodings(photo_ricardo)[0]
encoding_kate = face_recognition.face_encodings(photo_kate)[0]

known_face_encodings = [
    encoding_paula,
    encoding_ricardo,
    encoding_kate
]

known_face_labels = [
    "Paula",
    "Ricardo",
    "Kate Middleton"
]

In [ ]:
# Test the trained face recognition model on a sample image containing multiple people.

photo_test_path = "test\\photo_test.jpg"
photo_test = face_recognition.load_image_file(photo_test_path)

# Detects each face from the input image.
face_test_detections = face_recognition.face_locations(photo_test)
# Extracts face encodings.
face_test_encodings = face_recognition.face_encodings(photo_test, face_test_detections)

# Now, we are going to compare each detected face with our known faces.
# We are making the bounding box in the original image.
image = cv2.imread(photo_test_path)

# Analysing each face in the photo.
for (top, right, bottom, left), face_test_encodings in zip(face_test_detections, face_test_encodings):
    matches = face_recognition.compare_faces(known_face_encodings, face_test_encodings)

    # If a match is found, identify the person.
    name = "Unknown" 
    if True in matches:
        first_corresponding_index = matches.index(True)
        name = known_face_labels[first_corresponding_index]

    # Draw a bounding box around the detected face.
    # The coordinates are (top, right, bottom, left)
    cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2) # Green rectangle with thickness 2

    # Writing the name near to the face.
    # The label position is (left, top - 10) in order to stay above the rectangle.
    cv2.putText(image, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

if not os.path.exists("output"):
    os.makedirs("output")

success = cv2.imwrite("output\\photo_test_result.jpg", image)

cv2.imshow("Face recognition result", image)
# Wait until a key was pressed to close the window.
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Face recognition process finished!")

Face recognition process finished!
